# Convergent Strategy Exploration with defined Strategies

In [28]:
# Imports Chapter 1-10
from chapter1 import pd_readcsv, BUSINESS_DAYS_IN_YEAR, calculate_stats
from chapter3 import standardDeviation
from chapter4 import (
    create_fx_series_given_adjusted_prices_dict,
    calculate_variable_standard_deviation_for_risk_targeting_from_dict,
    calculate_position_series_given_variable_risk_for_dict,
)

from chapter5 import calculate_perc_returns_for_dict_with_costs
from chapter8 import apply_buffering_to_position_dict

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## Strategy 10: Carry 

##### We can create a strategy that uses multiple carry spans to determine our carry forecasts. We then use a capping method to the raw forecast of the trend signal combined with capital and risk management to determine our average position size.

### Instrument Selection

<ol>
<li>SP500</li>
<li>GAS</li>
</ol>

### Spans of Trends
##### The spans Carver outlines in his book are 5, 20, 60, and 120. We will use this to start.

### Defining Strategy Relevant Functions
##### All functions relevant to the strategy are below

In [21]:
INSTRUMENT_LIST = ["sp500", "gas"]

In [22]:
def get_data_dict_with_carry(instrument_list: list = None):
    
    if instrument_list is None:
        instrument_list = INSTRUMENT_LIST

    all_data = dict(
        [
            (instrument_code, pd_readcsv("%s.csv" % instrument_code))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )

    carry_data = dict(
        [
            (instrument_code, pd_readcsv("%s_carry.csv" % instrument_code))
            for instrument_code in instrument_list
        ]
    )


    return adjusted_prices, current_prices, carry_data

In [23]:
# Takes in adjusted prices dictionary, a dictionary containing standard deviations, a dictionary contiaing average
# position in contracts, carry prices, and a list of carry spans
# Main function is to calcluate position data for each instrument and the results are stored in a dict
def calculate_position_dict_with_multiple_carry_forecast_applied(
    adjusted_prices_dict: dict,
    std_dev_dict: dict,
    average_position_contracts_dict: dict,
    carry_prices_dict: dict,
    carry_spans: list,
) -> dict:

    list_of_instruments = list(adjusted_prices_dict.keys())
    position_dict_with_carry = dict(
        [
            (
                instrument_code,
                calculate_position_with_multiple_carry_forecast_applied(
                    average_position=average_position_contracts_dict[instrument_code],
                    stdev_ann_perc=std_dev_dict[instrument_code],
                    carry_price=carry_prices_dict[instrument_code],
                    carry_spans=carry_spans,
                ),
            )
            for instrument_code in list_of_instruments
        ]
    )

    return position_dict_with_carry

# Calcualtes single position function with a combined forecast
def calculate_position_with_multiple_carry_forecast_applied(
    average_position: pd.Series,
    stdev_ann_perc: standardDeviation,
    carry_price: pd.DataFrame,
    carry_spans: list,
) -> pd.Series:

    forecast = calculate_combined_carry_forecast(
        stdev_ann_perc=stdev_ann_perc,
        carry_price=carry_price,
        carry_spans=carry_spans,
    )

    return forecast * average_position / 10


def calculate_combined_carry_forecast(
    stdev_ann_perc: standardDeviation,
    carry_price: pd.DataFrame,
    carry_spans: list,
) -> pd.Series:

    all_forecasts_as_list = [
        calculate_forecast_for_carry(
            stdev_ann_perc=stdev_ann_perc,
            carry_price=carry_price,
            span=span,
        )
        for span in carry_spans
    ]

    ### NOTE: This assumes we are equally weighted across spans
    ### eg all forecast weights the same, equally weighted
    all_forecasts_as_df = pd.concat(all_forecasts_as_list, axis=1)
    average_forecast = all_forecasts_as_df.mean(axis=1)

    ## apply an FDM
    rule_count = len(carry_spans)
    FDM_DICT = {1: 1.0, 2: 1.02, 3: 1.03, 4: 1.04}
    fdm = FDM_DICT[rule_count]

    scaled_forecast = average_forecast * fdm
    capped_forecast = scaled_forecast.clip(-20, 20)

    return capped_forecast

# Scale so the scaled forecast has an absolute value of 10
# Finally, cap our forecast
def calculate_forecast_for_carry(
    stdev_ann_perc: standardDeviation, carry_price: pd.DataFrame, span: int
):

    smooth_carry = calculate_smoothed_carry(
        stdev_ann_perc=stdev_ann_perc, carry_price=carry_price, span=span
    )
    scaled_carry = smooth_carry * 30
    capped_carry = scaled_carry.clip(-20, 20)

    return capped_carry


# Calculates smoothed carry for each span
# If we take the average carry over a long enough period, it will smooth our any seasonal effects
def calculate_smoothed_carry(
    stdev_ann_perc: standardDeviation, carry_price: pd.DataFrame, span: int
):

    risk_adj_carry = calculate_vol_adjusted_carry(
        stdev_ann_perc=stdev_ann_perc, carry_price=carry_price
    )

    smooth_carry = risk_adj_carry.ewm(span).mean()

    return smooth_carry


# Returns risk adjusted price by computing annualized carry and dividing it by the annual price volatility (standard deviation)
def calculate_vol_adjusted_carry(
    stdev_ann_perc: standardDeviation, carry_price: pd.DataFrame
) -> pd.Series:
    
    # Returns annualized carry
    ann_carry = calculate_annualised_carry(carry_price)
    # Returns annual standard deviation of returns 
    ann_price_vol = stdev_ann_perc.annual_risk_price_terms()

    # gets risk adjusted carry
    risk_adj_carry = ann_carry.ffill() / ann_price_vol.ffill()
    # Carry = Annualized raw carry / (Stdev x 16)

    return risk_adj_carry

# Returns annualized carry by getting raw carry and dividing by difference in contracts 
def calculate_annualised_carry(
    carry_price: pd.DataFrame,
):

    ## Will be reversed if price_contract > carry_contract
    # Raw carry = Price of nearer futures contract - price of currently held contract 
    raw_carry = carry_price.PRICE - carry_price.CARRY
    # Calculates the difference of time between expiries as a fraction of a year
    # Absolute value of difference between contracts divided by 12
    contract_diff = _total_year_frac_from_contract_series(
        carry_price.CARRY_CONTRACT
    ) - _total_year_frac_from_contract_series(carry_price.PRICE_CONTRACT)
    
    # Example 
    # total_year_frac_from_contract_series(19830300) - total_year_from_contract_series(19821200)
    # 1983.25 - 1983 = 0.25

    # Annualized the raw carry
    ann_carry = raw_carry / contract_diff
    
    return ann_carry

# Takes the input x and divides it by 10,000 using floor d
def _total_year_frac_from_contract_series(x):
    years = _year_from_contract_series(x)
    month_frac = _month_as_year_frac_from_contract_series(x)

    return years + month_frac

# _total_year_frac_from_contract_series(19830300)
# years = 1983
# month = 0.25
# return 1983.25

# _total_year_frac_from_contract_series(19821200)
# years = 1982
# month_frac = 0.1
# return 1983

# Takes the input x and divides it by 10,000 using floor division
# For example, _year_from_contract_series(20231107), would return 2023
def _year_from_contract_series(x):
    return x.floordiv(10000)
# _year_from_contract_series(19830300) = 1983
# _year_from_contract_series(19821200) = 1982

# This function is intended to calculate the month as a fraction of a year
# First extracts the month component from the contract series then divides by 
# 12 to express as a fraction of a year
def _month_as_year_frac_from_contract_series(x):
    return _month_from_contract_series(x) / 12.0
# _month_as_year_frac_from_contract_series(19830300) = 3/12 = 0.25
# _month_as_year_frac_from_contract_series(19821200) = 12/12 = 1

# Intended to extract the month component from identifier, x. It does so by using 
# modulo and division. 
# Takes the input 'x' and calculates the remainder when divided by 10,000. It effectively extracts 
# the last four digits of 'x' which are assumed to represent the month and day information
# The function then divides this result by 100 to convert the month from a 4-digit representation 
# to a fractional value
# Example, if the last 4 digits are 0603, dividing by 100 results in 6.03, representting the month and day 
# as a fractional value 
# We are dealing with 12, 03, 06, 09 so we don't have to worry about days really
def _month_from_contract_series(x):
    return x.mod(10000) / 100.0
# _month_from_contract_series(19830300) = 0300 / 100 = 3
# _month_from_contract_series(19821200) = 1200 / 100 = 12

In [36]:
carry_spans = [5, 20, 60, 120]

In [37]:
    (
        adjusted_prices_dict,
        current_prices_dict,
        carry_prices_dict,
    ) = get_data_dict_with_carry()


    # Algorithm that gives Dictionary for multipliers that automatically
    multipliers = dict(sp500=5, gas=10000)
    risk_target_tau = 0.2
    fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict)

    capital = 500000
    
    # Idm will depend on how many instruments we use. This is hardcoded for now
    idm = 1.5
    # 3.5 for idm 
    # Should get a SR similar to Carver
    # Replicate 
    # Add it to each asset 
    
    # We will use dynamic optimization for this 
    instrument_weights = dict(sp500=0.5, gas=0.5)
    # Just put a dollar for now
    cost_per_contract_dict = dict(sp500=0.875, gas=15.3)

    std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
        adjusted_prices=adjusted_prices_dict, current_prices=current_prices_dict
    )
    
    average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )
        

##### This code established risk targets, capital, and other hardcoded numbers as well as bringing in historical data

In [38]:
    
    position_contracts_dict = (
        calculate_position_dict_with_multiple_carry_forecast_applied(
            adjusted_prices_dict=adjusted_prices_dict,
            carry_prices_dict=carry_prices_dict,
            std_dev_dict=std_dev_dict,
            average_position_contracts_dict=average_position_contracts_dict,
            carry_spans=carry_spans,
        )
    )

    buffered_position_dict = apply_buffering_to_position_dict(
        position_contracts_dict=position_contracts_dict,
        average_position_contracts_dict=average_position_contracts_dict,
    )

    perc_return_dict = calculate_perc_returns_for_dict_with_costs(
        position_contracts_dict=buffered_position_dict,
        fx_series=fx_series_dict,
        multipliers=multipliers,
        capital=capital,
        adjusted_prices=adjusted_prices_dict,
        cost_per_contract_dict=cost_per_contract_dict,
        std_dev_dict=std_dev_dict,
    )


/Users/stefanmostovych/Documents/General/University of Florida/AlgoGators/Python/All/chapter8.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_position = use_optimal_position[0]
/Users/stefanmostovych/Documents/General/University of Florida/AlgoGators/Python/All/chapter8.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  top_pos=upper_buffer[idx],
/Users/stefanmostovych/Documents/General/University of Florida/AlgoGators/Python/All/chapter8.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

In [39]:
print(calculate_stats(perc_return_dict["sp500"]))

{'ann_mean': -0.013487680302810308, 'ann_std': 0.09475602254102886, 'sharpe_ratio': -0.1423411403425066, 'skew': 1.4892343758962205, 'avg_drawdown': 0.833165961377379, 'max_drawdown': 1.4058131342124087, 'quant_ratio_lower': 3.4512856631076607, 'quant_ratio_upper': 3.4512856631076607}


In [40]:
print(calculate_stats(perc_return_dict["gas"]))

{'ann_mean': 0.07186569691670172, 'ann_std': 0.23484831089910724, 'sharpe_ratio': 0.3060090006249856, 'skew': -0.18202106464427353, 'avg_drawdown': 0.3777125732811205, 'max_drawdown': 1.3498850801200195, 'quant_ratio_lower': 1.5716220385129391, 'quant_ratio_upper': 1.5716220385129391}


In [41]:
position_contracts_df = pd.DataFrame.from_dict(position_contracts_dict) 
position_contracts_df.dropna(inplace=True)
position_contracts_df.head()

,sp500,gas
1990-07-30,-310.940340,-82.590167
1990-07-31,-317.705377,-67.499346
1990-08-01,-325.902060,-78.210789
1990-08-02,-329.027721,-45.198364
1990-08-03,-311.389501,-25.922002


In [42]:
perc_return_df = pd.DataFrame.from_dict(perc_return_dict)
perc_return_df.dropna(inplace=True)
perc_return_df.head(10)

,sp500,gas
1990-07-30,-0.007838,-0.000095
1990-07-31,-0.000998,0.026510
1990-08-01,0.003323,0.012776
1990-08-02,0.010543,-0.056300
1990-08-03,0.021750,-0.079978
1990-08-06,0.036749,-0.048619
1990-08-07,-0.008149,0.045970
1990-08-08,-0.006721,0.014400
1990-08-10,0.011603,-0.010800
1990-08-14,-0.002688,0.007200


### Plotting Histograms of Returns
##### Plotting the histogram of returns for sp500 and gas

In [43]:
# Plotting percentage returns of the SP500 and Gas futures
plt.figure(1)
plt.hist(perc_return_df["sp500"],bins=500)
plt.title("SP500 Returns")
plt.xlabel("Date")
plt.ylabel("Percentage Return")
plt.figure(2)
plt.hist(perc_return_df["gas"], bins=500)
plt.title("Gas Returns")
plt.xlabel("Date")
plt.ylabel("Percentage Return")

plt.show()